In [1]:
%cd ../..
from inference_diffusion import StyleTTS2

import librosa
import IPython.display as ipd
import torch.cuda

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

c:\code\StyleTTS2-lite


c:\code\StyleTTS2-lite\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
c:\code\StyleTTS2-lite\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\catto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Load G2P

If you did not use eSpeak for your language, please add your own G2P.

In [2]:
import sys
import phonemizer
if sys.platform.startswith("win"):
    try:
        from phonemizer.backend.espeak.wrapper import EspeakWrapper
        import espeakng_loader
        EspeakWrapper.set_library(espeakng_loader.get_library_path())
    except Exception as e:
        print(e)

def get_phoneme(text, lang):
    try:
        my_phonemizer = phonemizer.backend.EspeakBackend(language=lang, preserve_punctuation=True,  with_stress=True, language_switch='remove-flags')
        return my_phonemizer.phonemize([text])[0]
    except Exception as e:
        print(e)

In [3]:
from phonemizer.backend.espeak.wrapper import EspeakWrapper
import espeakng_loader
EspeakWrapper.set_library(espeakng_loader.get_library_path())

### Load models

In [4]:
config_path = "Configs/config.yaml"
models_path = "Models/Finetune/current_model.pth"

### Synthesize speech

Little Note: Reference audio has a huge impact on the result. It is best to select audio around 10s long and consistent in both tone and speed.

In [13]:
speaker = {
    "path": "./Demo/Audio/1_heart.wav",  #Ref audio path
    "speed": 1.0,                        #Speaking speed
}

max_samples = 24000*20 #max 20 seconds ref audio
print(speaker['path'])
wave, sr = librosa.load(speaker['path'], sr=24000)
audio, index = librosa.effects.trim(wave, top_db=30)
if sr != 24000:              audio = librosa.resample(audio, sr, 24000)
if len(audio) > max_samples: audio = audio[:max_samples]
display(ipd.Audio(audio, rate=24000, normalize=True))

./Demo/Audio/1_heart.wav


In [6]:
text = '''Đây là chuyến thăm Việt Nam đầu tiên của Tổng thống Emmanuel Macron kể từ khi ông nhậm chức vào năm 2017 và tái đắc cử vào năm 2022.'''

| Parameter      | Type   | Description                                                                 | Performance Impact                                |
|----------------|--------|-----------------------------------------------------------------------------|----------------------------------------------------|
| `avg_style`     | BOOL   | Split the reference audio and calculate the average speaking style.        | Higher computation during style extraction         |
| `denoise`       | FLOAT  | Adjusts denoiser strength; range [0, 1].                                   | Additional computation for style processing        |
| `stabilize`     | BOOL   | Stabilizes speaking speed for long-form synthesis.                         | Slight additional computation                      |
| `n_merge`       | INT    | Avoids short sentences by merging if words < `n_merge`.                   | Higher VRAM usage as value increases               |


In [7]:
model = StyleTTS2(config_path, 
                  models_path
                ).eval().to(device)


Found: 189 symbols


c:\code\StyleTTS2-lite\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
c:\code\StyleTTS2-lite\.venv\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


decoder : 54289492
predictor : 16194612
text_encoder : 5612032
style_encoder : 13845440

Total : 89941576


In [18]:
avg_style         = False
denoise           = 0.0
stabilize         = False    
n_merge           = 99999 

steps             = 5
embedding_scale   = 1.0
with torch.no_grad():
    phonemes = get_phoneme(text=text, lang="vi")
    styles  = model.get_styles(speaker, denoise, avg_style)
    r       = model.generate(phonemes, styles, steps, embedding_scale, n_merge, stabilize, use_diffusion=False)

print('Synthesized:')
display(ipd.Audio(r, rate=24000, normalize=False))

Computing the style for: ./Demo/Audio/1_heart.wav
Generating Audio...
Synthesized:


In [9]:
# avg_style         = False
# denoise           = 0.0
# stabilize         = False    
# n_merge           = 20
# with torch.no_grad():
#     phonemes = get_phoneme(text=text, lang="vi")
#     styles  = model.get_styles(speaker, denoise, avg_style)
#     r       = model.generate(phonemes, styles, n_merge, stabilize)

# print('Synthesized:')
# display(ipd.Audio(r, rate=24000, normalize=True))

Optional: The styles tensor can be saved and re-use later for faster generation time.

In [10]:
# model.save_styles(save_dir="./style1.pt")
# model.load_styles(save_dir="./style1.pt")

In [11]:
# with torch.no_grad():
#     phonemes = get_phoneme(text=text, lang="en-us")
#     styles  = model.get_styles(speaker, load_styles=True)
#     r       = model.generate(phonemes, styles, stabilize, n_merge)

# print('Synthesized:')
# display(ipd.Audio(r, rate=24000, normalize=True))